In [8]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy


In [12]:
df = pd.read_csv("/home/antoine/mlops_reco_movies/airflow/data/raw/bronze/genome-tags.csv")
df.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [14]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

Missing values in each column:
tagId    0
tag      0
dtype: int64


In [15]:
df = df.rename(columns={"tagId":"tag_id"})
df.head()

,tag_id,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [6]:
df["clean_tag"] = (
            df["tag"]
            .str.replace(r"[\(\)\[\]\d+]", "", regex=True)
            .str.strip()
            .str.lower()
        )

df.head()

,tag_id,tag,clean_tag
0,1,007,
1,2,007 (series),series
2,3,18th century,th century
3,4,1920s,s
4,5,1930s,s


In [ ]:
nlp = spacy.load("en_core_web_sm")

def clean_tag(tag: str) -> str:
    doc = nlp(tag)
    return " ".join(
        token.text.lower()
        for token in doc
        if not token.is_punct and not token.is_stop
    )

In [17]:
df["clean_tag"]= df["tag"].apply(clean_tag)

df.head()

,tag_id,tag,clean_tag
0,1,007,007
1,2,007 (series),007 series
2,3,18th century,18th century
3,4,1920s,1920s
4,5,1930s,1930s
